In [5]:
import sys
import os
import json
import re

src_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_directory)

from llm_handler import *
from fine_tuner import *

In [ ]:
rag_knowledge_base_path = "../../data/cdm_samples_json"

llm_handler = FineTuner(
    model_id="meta-llama/Llama-3.2-3B-Instruct",
    token="huggingface access token",
    json_folder="../../data/old_fine_tuning_data/cdm_examples", 
    txt_folder="../../data/old_fine_tuning_data/contract_descriptions"
)

llm_handler.initialize_rag(
    documents_path=rag_knowledge_base_path,
    embed_model_name="BAAI/bge-small-en-v1.5",
    top_k=5,
    similarity_cutoff=0.6
)

==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.016 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 9.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
meta-llama/Llama-3.2-3B-Instruct initialized for fine-tuning!
LLM is explicitly disabled. Using MockLLM.
Initialized RAG with 856 documents and embedding model: BAAI/bge-small-en-v1.5


In [7]:
llm_handler.fine_tune(max_steps=100)

Map: 100%|██████████| 542/542 [00:00<00:00, 24161.56 examples/s]


Dataset Preparation Completed!
Training...


Map (num_proc=2): 100%|██████████| 542/542 [00:02<00:00, 235.57 examples/s]
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 542 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.296100
2,1.428500
3,1.471200
4,1.363800
5,1.507700
6,1.149000
7,1.294600
8,1.218000
9,1.219500
10,1.208500


Training Completed!


In [10]:

contract_descriptions_path = 'test_data'

contracts = []

for filename in os.listdir(contract_descriptions_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(contract_descriptions_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            contracts.append({'name': os.path.splitext(filename)[0], 'description': file.read().strip()})

In [11]:
def extract_json(string):
    print("Curr Json String ---> ")
    print(string)
    match = re.search(r'(\{.*\})', string, re.DOTALL)
    if match:
        json_string = match.group(1)
        
        try:
            json_obj = json.loads(json_string)
            return json_obj
        except Exception:
            return None
    else:
        return None

def is_json(string):
    try:
        json.loads(string)
        return True
    except Exception:
        return False

In [ ]:
# rag_knowledge_base_path = "../../data/cdm_samples_json"

# llm_handler_rag = FineTuner(
#     model_id="meta-llama/Llama-3.2-3B-Instruct",
#     token="huggingface access token"
# )

# llm_handler_rag.initialize_rag(
#     documents_path=rag_knowledge_base_path,
#     embed_model_name="BAAI/bge-small-en-v1.5",
#     top_k=5,
#     similarity_cutoff=0.6
# )

cdms = []

for contract in contracts:

    exclude = ['jpmorgan_interest_rate_swap', 'rbccapital_swap', 'bofa_equity_option', 'bofa_equity_swap_confirmation', 'bofa_interest_rate_swap']

    if contract['name'] in exclude:
        continue
    
    print("Working on ----> ", contract['name'], "\n")
    while(True):
        contract_summary = llm_handler.generate("Summarize the following contract in AT MOST 500 words : " + contract["description"])
        if len(contract_summary) < 5000:
            print("Contract Summary for RAG context retrieval is generated!")
            break

    # RAG context retrieval
    rag_context = llm_handler.get_context_using_rag(f"Find CDM representations that best capture the information from a derivatives contract like the following:\n\n{contract_summary}")

    # Prepare prompts
    basic_prompt = f"""
    Represent the following derivatives contract in a complete ISDA CDM JSON format, adhering strictly to the CDM schema. Ensure all key details are covered and follow the schema definitions exactly:

    Derivatives Contract:
    {contract['description']}

    Please ensure:
    1. The output is **concise** but **complete**, following the ISDA CDM dictionary.
    2. **Ignore** any 'meta' or 'globalKey' in your generated response. 
    3. **Only** provide the JSON representation, with no additional text or explanations.
    4. If the content exceeds the token limit, **truncate** appropriately while still providing a **valid** and complete JSON structure.
    
    Output the **final JSON** below.
    """

    rag_prompt = f"""
    Represent the following derivatives contract in a complete ISDA CDM JSON format, adhering strictly to the CDM schema. Ensure all key details are covered and follow the schema definitions exactly:

    Derivatives Contract:
    {contract['description']}

    Relevant CDM Examples for Reference:
    {rag_context}

    Please ensure:
    1. The output is **concise** but **complete**, following the ISDA CDM dictionary.
    2. **Ignore** any 'meta' or 'globalKey' in your generated response. 
    3. **Only** provide the JSON representation, with no additional text or explanations.
    4. If the content exceeds the token limit, **truncate** appropriately while still providing a **valid** and complete JSON structure.
    
    Output the **final JSON** below.
    """


    # Saving the responses to JSON files if they don't already exist
    cdm_file_path = f"test_data/generated_cdms/Fine-tuned-Llama-3.2-3B-Instruct/{contract['name']}_cdm.json"
    cdm_rag_file_path = f"test_data/generated_cdms/Fine-tuned-Llama-3.2-3B-Instruct/{contract['name']}_cdm_rag.json"

    if not os.path.exists(cdm_file_path):
        conf = ""
        while(True):
            cdm_response = extract_json(llm_handler.generate(basic_prompt+conf))
            if cdm_response:
                print("Generated a correct JSON!!")
                break
            else:
                print(cdm_response, "\n")
                conf = "\n\nNote that your previous response didn't generate a COMLETE JSON probably because of the token limit. Make sure to generate a COMPLETE JSON this time even if you need to ignore some less important data."
                print("Generated CDM is not a correct JSON. Generating again...")

        with open(cdm_file_path, 'w') as file:
            json.dump(cdm_response, file, indent=4)
        print(f"Saved CDM response to {cdm_file_path}")

    if not os.path.exists(cdm_rag_file_path):
        conf = ""
        while(True):
            cdm_with_rag_response = extract_json(llm_handler.generate(rag_prompt+conf))
            if cdm_with_rag_response:
                print("(RAG) Generated a correct JSON!!")
                break
            else:
                print(cdm_with_rag_response, "\n")
                conf = "\n\nNote that your previous response didn't generate a COMLETE JSON probably because of the token limit. Make sure to generate a COMPLETE JSON this time even if you need to ignore some less important data."
                print("(RAG) Generated CDM is not a correct JSON. Generating again...")

        with open(cdm_rag_file_path, 'w') as file:
            json.dump(cdm_with_rag_response, file, indent=4)
        print(f"Saved CDM_RAG response to {cdm_rag_file_path}")


    # Print or process the responses
    print("CDM Response without RAG:")
    print(cdm_response)
    print("\nCDM Response with RAG:")
    print(cdm_with_rag_response)

    cdms.append(
        {
            "name": contract['name'],
            "description": contract['description'],
            "cdm": cdm_response,
            "cdm_rag": cdm_with_rag_response
        }
    )

Working on ---->  jpmorgan_interest_rate_swap 

Contract Summary for RAG context retrieval is generated!
Curr Json String ---> 
{"trade": {"tradeIdentifier": [{"issuerReference": {"globalReference": "b1b1c9", "externalReference": "party1"}, "assignedIdentifier": [{"identifier": {"value": "JPM-1234", "meta": {"scheme": "http://www.JPMorgan.com/com-trade-id"}}}], "meta": {"globalKey": "c0b7d6a1"}}, {"issuerReference": {"globalReference": "b1b1c9", "externalReference": "party1"}, "assignedIdentifier": [{"identifier": {"value": "JPM-1234", "meta": {"scheme": "http://www.JPMorgan.com/com-trade-id"}}}], "meta": {"globalKey": "c0b7d6a1"}}], "tradeDate": {"value": "2021-04-07", "meta": {"globalKey": "3f2907"}}, "tradableProduct": {"product": {"contractualProduct": {"productTaxonomy": [{"source": "ISDA", "productQualifier": "InterestRate_IRSwap_Basis"}], "economicTerms": {"payout": {"interestRatePayout": [{"payerReceiver": {"payer": "Party1", "receiver": "Party2"}, "priceQuantity": {"quantitySc

KeyboardInterrupt: 

In [ ]:
from evaluate import *

cdm_schema_path = "../cdm_schema/"
cdm_repo_path = "../cdm_schema/cdm_schema_json"

cdms = []

for contract in contracts:

    # exclude = ['bofa_interest_rate_swap', 'bofa_equity_option', 'jpmorgan_interest_rate_swap']
    
    # if contract['name'] in exclude:
    #     continue

    cdm_file_path = f"test_data/generated_cdms/Fine-tuned-Llama-3.2-3B-Instruct/{contract['name']}_cdm.json"
    cdm_rag_file_path = f"test_data/generated_cdms/Fine-tuned-Llama-3.2-3B-Instruct/{contract['name']}_cdm_rag.json"

    if os.path.exists(cdm_file_path):
        with open(cdm_file_path, 'r') as cdm_file:
            cdm = json.load(cdm_file)

    if os.path.exists(cdm_rag_file_path):
        with open(cdm_rag_file_path, 'r') as cdm_rag_file:
            cdm_rag = json.load(cdm_rag_file)

    syntactic_correctness, schema_adherence = evaluate_cdm(cdm, cdm_schema_path, cdm_repo_path)
    rag_syntactic_correctness, rag_schema_adherence = evaluate_cdm(cdm_rag, cdm_schema_path, cdm_repo_path)

    semantic_coverage = get_coverage_score(get_coverage(llm_handler, contract['description'], cdm))
    rag_semantic_coverage = get_coverage_score(get_coverage(llm_handler, contract['description'], cdm_rag))

    print("\n", contract['name'])
    print("-----------------------\n")
    print(syntactic_correctness)
    print(schema_adherence)
    print("\n")
    print(rag_syntactic_correctness)
    print(rag_schema_adherence)
    print("\n")
    print(semantic_coverage)
    print(rag_semantic_coverage)
    print("\n")

    print("======================\n")
    # break